Generic Types, Traits and Lifetime
---

Every programming language has tools for effectively handling the duplication of  
concepts. In Rust, one such tool is generics: abstract stand-ins for concrete  
types or other properties. We can express the behavior of generics or how they  
relate to other generics without knowing what will be in their place when  
compiling and running the code.  

Functions can take parameters of some generic type, instead of a concrete type  
like i32 or String, in the same way a function takes parameters with unknown  
values to run the same code on multiple concrete values. In fact, we’ve already  
used generics in Chapter 6 with Option<T>, Chapter 8 with Vec<T> and  
HashMap<K, V>, and Chapter 9 with Result<T, E>. In this chapter, you’ll explore  
how to define your own types, functions, and methods with generics!  

First, we’ll review how to extract a function to reduce code duplication. We’ll  
then use the same technique to make a generic function from two functions that  
differ only in the types of their parameters. We’ll also explain how to use  
generic types in struct and enum definitions.  

Then you’ll learn how to use traits to define behavior in a generic way. You  
can combine traits with generic types to constrain a generic type to accept only  
those types that have a particular behavior, as opposed to just any type.  

Finally, we’ll discuss lifetimes: a variety of generics that give the compiler  
information about how references relate to each other. Lifetimes allow us to  
give the compiler enough information about borrowed values so that it can ensure  
references will be valid in more situations than it could without our help.  

### Removing Duplication by Extracting a Function

Generics allow us to replace specific types with a placeholder that represents  
multiple types to remove code duplication. Before diving into generics syntax,  
then, let’s first look at how to remove duplication in a way that doesn’t  
involve generic types by extracting a function that replaces specific values  
with a placeholder that represents multiple values. Then we’ll apply the same  
technique to extract a generic function! By looking at how to recognize  
duplicated code you can extract into a function, you’ll start to recognize  
duplicated code that can use generics.

We begin with the short program in Listing 10-1 that finds the largest number in  
a list.

> Filename: src/main.rs

In [2]:
{   // begin block closure
    let number_list = vec![34, 50, 25, 100, 65]; // number_list comes into scope and
                                                 // gets ownership of the vector, so
                                                 // is responsible for releasing it
                                                 // - `number_list` is stored on the stack
                                                 // it contains meta data about the vector
                                                 // wrt to length and capacity of the ACTUAL
                                                 // VALUE on the HEAP

    let mut largest = &number_list[0];  // largest comes into scope
                                        // we don't take ownership of the vector,
                                        // we borrow a reference to it
                                        // largest stores this reference on the stack

    for number in &number_list {    // looping over an immutable reference to &number_list
                                    // - by using a reference we AVOID loop taking ownership of
                                    // the number_list vector, so we can use it later (println!)
                                    // - the loop will create an immutable reference `number` to
                                    // each element in the vector
        if number > largest {       // we compare `number` to `largest`
                                    // - because both are references, Rust automatically
                                    // dereferences each value for comparison thanks to `deref` trait
            largest = number;       // if current `number` is larger than `largest`
                                    // - we update the mutable reference `largest` to point to
                                    // the current `number` reference
                                    // - we DO NOT move OWNERSHIP, instead we only update the REFERENCE
        }
    }

    println!("The largest number is {}", largest);

}   // exit block closure
    // largest goes out of scope, and are released : it is a reference so no heap changes
    // number_list goes out of scope, and is released from memory
    // - the vector meta data wrt to length and capacity is released from the stack
    // - Rust automatically calls the `drop` function to release the VECTOR's VALUE
    // from the HEAP

The largest number is 100


()

> Listing 10-1: Finding the largest number in a list of numbers

We store a list of integers in the variable `number_list` and place a reference to  
the first number in the list in a variable named `largest`. We then iterate  
through all the numbers in the list, and if the current number is greater than  
the number stored in `largest`, replace the reference in that variable. However,  
if the current number is less than or equal to the largest number seen so far,  
the variable doesn’t change, and the code moves on to the next number in the  
list. After considering all the numbers in the list, `largest` should refer to  
the largest number, which in this case is 100.  

We've now been tasked with finding the largest number in two different lists of  
numbers. To do so, we can choose to duplicate the code in Listing 10-1 and use  
the same logic at two different places in the program, as shown in Listing 10-2.  

> Filename: src/main.rs

In [4]:
{   // Bloc comes into scope
    let number_list = vec![34, 50, 25, 100, 65]; // (a) shadowed

    let mut largest = &number_list[0];  // (a) shadowed

    for number in &number_list { // number is an immutable reference to each element in the vector
        if number > largest {    // rust automatically dereferences `number` and `largest`
                                 // to compare because they are BOTH references
                                 // deref coercion REQUIRES that BOTH are `number` and `largest`
                                 // are references, else YOU must manually * dereference
            largest = number;    // this is NOT DEREF COERCION ...
                                 // but because both are &i32 references, they can DIRECTLY be
                                 // ASSIGNED without DEREFERENCING
        }
    }

    println!("The largest number is {}", largest);

    let number_list = vec![102, 34, 6000, 89, 54, 2, 43, 8]; // (b) shadowing

    let mut largest = &number_list[0];  // (b) shadowing

    for number in &number_list{
        if number > largest {
            largest = number;
        }
    }

    println!("The largest number is {}", largest);
}   // Block leaves scope
    // on block end, Rust automatically deallocates memory and cleans up variables
    // that go out of scope ... including their shadows, here's the order of op:
    // - `largest` (b) goes out of scope, it is a reference and are popped of the stack
    // no heap changes are applied, the memory it refs is owned by `number_list` (b)
    // vector and will get cleaned up when that is released
    // - `number_list` (b) goes out of scope, the vector ref containing meta data
    // wrt to length and capacity is popped off the stack
    // the actual VECTOR VALUE is released from the HEAP when RUST calls `drop`
    // - `largest` (a) goes out of scope, it is a reference and are popped of the stack
    // no heap changes are applied, the memory it refs is owned by `number_list` (a)
    // vector and will get cleaned up when that is released
    // - `number_list` (a) goes out of scope, the vector ref containing meta data
    // wrt to length and capacity is popped off the stack
    // the actual VECTOR VALUE is released from the HEAP when RUST calls `drop`



The largest number is 100
The largest number is 6000


()

> Listing 10-2: Code to find the largest number in two lists of numbers

Although this code works, duplicating code is tedious and error prone. We also  
have to remember to **`update the code in multiple places`** when we want to  
change it.

To eliminate this duplication, we’ll create an abstraction by defining a  
function that operates on any list of integers passed in a parameter. This  
solution makes our code clearer and lets us express the concept of finding the  
largest number in a list abstractly.

In Listing 10-3, we extract the code that finds the largest number into a  
function named largest. Then we call the function to find the largest number  
in the two lists from Listing 10-2. We could also use the function on any other  
list of i32 values we might have in the future.

> Filename: src/main.rs



In [5]:
fn largest(list: &[i32])-> &i32 {
    let mut largest = &list[0];

    for number in list {
        if number > largest {
            largest = number;
        }
    }

    largest
}

{
    let number_list = vec![34, 50, 25, 100, 65];
    let result = largest(&number_list);
    println!("The largest number is {}", result);

    let number_list = vec![102, 34, 6000, 89, 54, 2, 43, 8];
    let result = largest(&number_list);
    println!("The largest number is {}", result);
}

The largest number is 100
The largest number is 6000


()

> Listing 10-3: Abstracted code to find the largest number in two lists

The largest function has a parameter called list, which represents any concrete  
slice of i32 values we might pass into the function. As a result, when we call  
the function, the code runs on the specific values that we pass in.



In summary, here are the steps we took to change the code from Listing 10-2 to  
Listing 10-3:

- Identify duplicate code.  
- Extract the duplicate code into the body of the function and specify the  
inputs and return values of that code in the function signature.  
- Update the two instances of duplicated code to call the function instead.  

Next, we’ll use these same steps with generics to reduce code duplication. In  
the same way that the function body can operate on an abstract list instead of  
specific values, generics allow code to operate on abstract types.  

For example, say we had two functions: one that finds the largest item in a  
slice of i32 values and one that finds the largest item in a slice of char  
values. How would we eliminate that duplication? Let’s find out!  

Generic Data Types
---

We use generics to create definitions for items like function signatures or  
structs, which we can then use with many different concrete data types. Let’s  
first look at how to define functions, structs, enums, and methods using  
generics. Then we’ll discuss how generics affect code performance.  

### In Function Definitions

When defining a function that uses generics, we place the generics in the  
signature of the function where we would usually specify the data types of the  
parameters and return value. Doing so makes our code more flexible and provides  
more functionality to callers of our function while preventing code duplication.  

Continuing with our largest function, Listing 10-4 shows two functions that both  
find the largest value in a slice. We'll then combine these into a single  
function that uses generics.  

> Filename: src/main.rs

In [10]:
fn largest_i32(list: &[i32])-> &i32 {
    let mut largest = &list[0];

    for item in list{
        if item > largest{
            largest = item;
        }
    }
    largest
}

fn largest_char(list: &[char])-> &char{
    let mut largest = &list[0];

    for item in list{
        if item > largest{
            largest = item;
        }
    }
    largest
}

{
    let n_list = vec![34, 50, 25, 100, 65];
    let result = largest_i32(&n_list);
    println!("The largest number is {}", result);

    let n_list = vec!['y', 'm', 'a', 'q'];  // shadowing allows swithc from i32 to char
    let result = largest_char(&n_list);     // same shaodow to re-use the same value names
    println!("The largest char is {}", result);
}

The largest number is 100
The largest char is y


()

> Listing 10-4: Two functions that differ only in their names and the types in  
their signatures

- The largest_i32 function is the one we extracted in Listing 10-3 that finds  
the largest i32 in a slice. 
- The largest_char function finds the largest char in a slice. 

> The function bodies have the same code, so let’s eliminate the duplication by  
introducing a generic type parameter in a single function.

To parameterize the types in a new single function, we need to name the type  
parameter, just as we do for the value parameters to a function. You can use any  
identifier as a type parameter name. But we’ll use T because, by convention,  
type parameter names in Rust are short, often just a letter, and Rust’s  
type-naming convention is UpperCamelCase. Short for “type,” T is the default  
choice of most Rust programmers.

When we use a parameter in the body of the function, we have to declare the  
parameter name in the signature so the compiler knows what that name means.  
Similarly, when we use a type parameter name in a function signature, we have  
to declare the type parameter name before we use it. To define the generic  
largest function, place type name declarations inside angle brackets, <>,  
between the name of the function and the parameter list, like this:

In [15]:
fn largest<T>(list:&[T])-> Option<&T> { None }

We read this definition as: the function largest is generic over some type T.  
This function has one parameter named list, which is a slice of values of type  
T. The largest function will return a reference to a value of the same type T.  

Listing 10-5 shows the combined largest function definition using the generic  
data type in its signature. The listing also shows how we can call the function  
with either a slice of i32 values or char values. Note that this code won’t  
compile yet, but we’ll fix it later in this chapter.

> Filename: src/main.rs

In [19]:
// restrict T to types that implement the PartialOrd trait
// - both i32 and char implement PartialOrd and this will allow code to compile
// fn largest<T: std::cmp::PartialOrd>(list:&[T])->&T{
fn largest<T>(list:&[T])->&T{
    let mut largest = &list[0];
    for item in list{
        if item > largest{
            largest = item;
        }
    }
    largest
}

{
    let n_list = vec![34, 50, 25, 100, 65];
    let result = largest(&n_list);
    println!("The largest number is {}", result);

    let n_list = vec!['y', 'm', 'a', 'q'];
    let result = largest(&n_list);
    println!("The largest char is {}", result);
}

Error: binary operation `>` cannot be applied to type `&T`

> Listing 10-5: The largest function using generic type parameters; this  
doesn’t yet compile

If we compile this code right now, we’ll get this error:

```sh
$ cargo run
   Compiling chapter10 v0.1.0 (file:///projects/chapter10)
error[E0369]: binary operation `>` cannot be applied to type `&T`
 --> src/main.rs:5:17
  |
5 |         if item > largest {
  |            ---- ^ ------- &T
  |            |
  |            &T
  |
help: consider restricting type parameter `T`
  |
1 | fn largest<T: std::cmp::PartialOrd>(list: &[T]) -> &T {
  |             ++++++++++++++++++++++

For more information about this error, try `rustc --explain E0369`.
error: could not compile `chapter10` due to previous error
```

The help text mentions std::cmp::PartialOrd, which is a trait, and we’re going  
to talk about traits in the next section. For now, know that this error states  
that the body of largest won’t work for all possible types that T could be.  
Because we want to compare values of type T in the body, we can only use types  
whose values can be ordered. To enable comparisons, the standard library has  
the std::cmp::PartialOrd trait that you can implement on types (see Appendix C  
for more on this trait). By following the help text's suggestion, we restrict  
the types valid for T to only those that implement PartialOrd and this example  
will compile, because the standard library implements PartialOrd on both  
i32 and char.

### In Struct Definitions

We can also define structs to use a generic type parameter in one or more fields  
using the <> syntax. Listing 10-6 defines a Point<T> struct to hold x and y  
coordinate values of any type.

> Filename: src/main.rs

In [26]:
#[derive(Debug)]
struct Point<T>{
    x:T,
    y:T,
}

{
    let integer = Point{x:5, y:10};
    let float = Point{x:1.0, y:4.0};
    println!("integer = {:?} float = {:?}", integer, float);
}

integer = Point { x: 5, y: 10 } float = Point { x: 1.0, y: 4.0 }


()

> I will explain the given Rust code snippet line by line, focusing on ownership,  
borrowing, and stack and heap changes:

```rust
#[derive(Debug)]               // Line 1
struct Point<T> {              // Line 2
    x: T,                      // Line 3
    y: T,                      // Line 4
}                              // Line 5

{                              // Line 6
    let integer = Point {      // Line 7
        x: 5,                  // Line 8
        y: 10,                 // Line 9
    };                         // Line 10
    let float = Point {        // Line 11
        x: 1.0,                // Line 12
        y: 4.0,                // Line 13
    };                         // Line 14
    println!("integer = {:?} float = {:?}", integer, float); // Line 15
}                              // Line 16
```

**Line 1**: We use the `derive` attribute to automatically implement the `Debug`  
trait for the `Point` struct. This allows instances of `Point` to be printed using the `{:?}` format specifier.

**Lines 2-5**: We define a generic struct called `Point` with a type parameter  
`T`. The struct has two fields, `x` and `y`, both of the generic type `T`.

**Line 6**: We start a new block scope. Variables created within this block will  
be dropped when the block ends at Line 16.

**Lines 7-10**: We create a new instance of `Point` with `i32` type called  
`integer`, with `x` set to 5 and `y` set to 10. The `integer` variable is stored  
on the stack, containing the `x` and `y` values directly since they are of type  
`i32`, which is a stack-allocated type.

**Lines 11-14**: We create a new instance of `Point` with `f64` type called  
`float`, with `x` set to 1.0 and `y` set to 4.0. Like `integer`, the `float`  
variable is stored on the stack, containing the `x` and `y` values directly  
since they are of type `f64`, which is also a stack-allocated type.

**Line 15**: We print the `integer` and `float` variables using the `println!`  
macro and the `Debug` formatting. Since both variables are still in scope and  
valid, this operation is allowed.

**Line 16**: This is the end of the block scope. At this point, Rust drops any  
remaining variables in reverse order of their creation. First, `float` is  
dropped, and then `integer` is dropped. No heap memory is deallocated in this  
example, as both instances of `Point` use stack-allocated types (`i32` and `f64`).

> In this code snippet, we don't have any explicit ownership transfers or  
borrowing. The ownership of the `Point` instances stays within their respective  
variables, `integer` and `float`, throughout the block scope.

> Listing 10-6: A Point<T> struct that holds x and y values of type T

The syntax for using generics in struct definitions is similar to that used in  
function definitions. First, we declare the name of the type parameter inside  
angle brackets just after the name of the struct. Then we use the generic type  
in the struct definition where we would otherwise specify concrete data types.

Note that because we’ve used only one generic type to define Point<T>, this  
definition says that the Point<T> struct is generic over some type T, and the  
fields x and y are both that same type, whatever that type may be. If we create  
an instance of a Point<T> that has values of different types, as in  
Listing 10-7, our code won’t compile.

> Filename: src/main.rs

In [27]:
struct Point<T> {
    x: T,
    y: T,
}

{
    let wont_work = Point{x:5, y:4.0};
}

Error: mismatched types

> Listing 10-7: The fields x and y must be the same type because both have the  
same generic data type T.

In this example, when we assign the integer value 5 to x, we let the compiler  
know that the generic type T will be an integer for this instance of Point<T>.  
Then when we specify 4.0 for y, which we’ve defined to have the same type as x, 
 we’ll get a type mismatch error like this:

```sh
$ cargo run
   Compiling chapter10 v0.1.0 (file:///projects/chapter10)
error[E0308]: mismatched types
 --> src/main.rs:7:38
  |
7 |     let wont_work = Point { x: 5, y: 4.0 };
  |                                      ^^^ expected integer, found floating-point number

For more information about this error, try `rustc --explain E0308`.
error: could not compile `chapter10` due to previous error
```
To define a Point struct where x and y are both generics but could have  
different types, we can use multiple generic type parameters. For example, in  
Listing 10-8, we change the definition of Point to be generic over types T and  
U where x is of type T and y is of type U.  

> Filename: src/main.rs


In [34]:
#[derive(Debug)]    // allows instnaces of Point to be printed using {:?} specifier
                    // by automatically implementing the std::fmt::Debug trait
                    // using the derive attribute

struct Point<T,U>{  // define a generic struct Point with type parameters T and U
    x:T,            // this supports x and y holding values of different types
    y:U,
}

{                   // new block scope begin
    let both_integer = Point{x:5, y:10};    // both_integer of type Point<i32, i32>
                                            // comes into scope ... and since 
                                            // - both i32 are are stack allocated
                                            // no heap allocation is required

    let both_float = Point{x:1.0, y:4.0};   // both_float of type Point<f32, f32>
                                            // comes into scope ... and since
                                            // - both f32 are stack allocated
                                            // no heap allocation is required

    let int_n_flt = Point{x:5, y:4.0};      // int_n_flt of type Point<i32, f32>
                                            // comes into scope ... and since
                                            // - i32 is stack allocated
                                            // - f32 is stack allocated
                                            // no heap allocation is required

    println!{"both_integer = {:?}", both_integer};

    //println!{"both_float = {:?}", both_float};    // uncommenting out this print
                                                    // changes nothing wrt to
                                                    // stack or heap allocation

    println!{"integer_and_float = {:?}", int_n_flt};
}                   // block scope end
                    // At this point all owned/local variables go out of scope
                    // and Rust will drop them in reverse order of their creation
                    // - int_n_flt is dropped first and popped off the stack
                    // - both_float is dropped next and popped off the stack
                    // - both_integer is finally dropped and popped off the stack
                    // Since all values are stack allocated, no heap allocation 
                    // clean up is necessary
                    // Lastly the whole block scope stack frame is popped off the
                    // stack and the program continues ... presumable back to
                    // the closure entry point


both_integer = Point { x: 5, y: 10 }
integer_and_float = Point { x: 5, y: 4.0 }


()

> Listing 10-8: A Point<T, U> generic over two types so that x and y can be  
values of different types

Now all the instances of Point shown are allowed! You can use as many generic  
type parameters in a definition as you want, but using more than a few makes  
your code hard to read. If you're finding you need lots of generic types in  
your code, it could indicate that your code needs restructuring into smaller  
pieces.

### In Enum Definitions

As we did with structs, we can define enums to hold generic data types in their  
variants. Let’s take another look at the Option<T> enum that the standard  
library provides, which we used in Chapter 6:

```rust
enum Option<T> {
    Some(T),    // Some Generic T Type
    None,       // None
}
```

This definition should now make more sense to you. As you can see, the Option<T>  
enum is generic over type T and has two variants: Some, which holds one value of  
type T, and a None variant that doesn’t hold any value. By using the Option<T>  
enum, we can express the abstract concept of an optional value, and because  
Option<T> is generic, we can use this abstraction no matter what the type of the  
optional value is.

Enums can use multiple generic types as well. The definition of the Result enum  
that we used in Chapter 9 is one example:

```rust
enum Result<T, E> {
    Ok(T),  // Ok of generic T Type
    Err(E), // Error of generic E Type 
}
```

The Result enum is generic over two types, T and E, and has two variants: 

- Ok, which holds a value of type T, and  
- Err, which holds a value of type E.  

This definition makes it convenient to use the Result enum anywhere we have an  
operation that might succeed (return a value of some type T) or fail (return an  
error of some type E). In fact, this is what we used to open a file in  
Listing 9-3, where T was filled in with the type std::fs::File when the file was  
opened successfully and E was filled in with the type std::io::Error when there  
were problems opening the file.

When you recognize situations in your code with multiple struct or enum  
definitions that differ only in the types of the values they hold, you can avoid  
duplication by using generic types instead.

### In Method Definitions

We can implement methods on structs and enums (as we did in Chapter 5) and use  
generic types in their definitions, too. Listing 10-9 shows the Point<T> struct  
we defined in Listing 10-6 with a method named x implemented on it.  

> Filename: src/main.rs

In [36]:
struct Point<T>{
    x:T,
    y:T,
}

impl<T> Point<T> {
    fn x(&self) -> &T{
        &self.x
    }
}

{
    let p = Point{x:5, y:10};
    println!("p.x = {}", p.x());
}

p.x = 5


()

> Listing 10-9: Implementing a method named x on the Point<T> struct that will  
return a reference to the x field of type T

Here, we’ve defined a method named x on Point<T> that returns a reference to the  
data in the field x.

Note that we have to declare T just after impl so we can use T to specify that  
we’re implementing methods on the type Point<T>. By declaring T as a generic  
type after impl, Rust can identify that the type in the angle brackets in Point  
is a generic type rather than a concrete type. We could have chosen a different  
name for this generic parameter than the generic parameter declared in the  
struct definition, but using the same name is conventional. Methods written  
within an impl that declares the generic type will be defined on any instance of  
the type, no matter what concrete type ends up substituting for the generic  
type. 

We can also specify constraints on generic types when defining methods on the  
type. We could, for example, implement methods only on Point<f32> instances  
rather than on Point<T> instances with any generic type. In Listing 10-10 we use  
the concrete type f32, meaning we don’t declare any types after impl.  

> Filename: src/main.rs

In [40]:
struct Point<X1, Y1>{
    x:X1,
    y:Y1,
}

impl<X1, Y1> Point<X1, Y1>{  // impl declares X1 and Y1 
    // X2 and Y2 aren't declared until they are used in the `mixup` method
    // - the point is generics can be declared at different levels as needed
    fn mixup<X2, Y2>(self, other: Point<X2, Y2>) -> Point<X1, Y2>{
        Point{
            x: self.x,
            y: other.y,
        }
    }
}

{
    let p1 = Point{x:5, y:10.4};
    let p2 = Point{x:"Hello", y:'c'};

    let p3 = p1.mixup(p2);

    println!("p3.x = {}, p3.y = {}", p3.x, p3.y);
}

p3.x = 5, p3.y = c


()

> Listing 10-11: A method that uses generic types different from its struct’s  
definition

In main, we’ve defined a Point that has an i32 for x (with value 5) and an f64  
for y (with value 10.4). The p2 variable is a Point struct that has a string  
slice for x (with value "Hello") and a char for y (with value c). Calling mixup  
on p1 with the argument p2 gives us p3, which will have an i32 for x, because x  
came from p1. The p3 variable will have a char for y, because y came from p2.  
The println! macro call will print p3.x = 5, p3.y = c.

The purpose of this example is to demonstrate a situation in which some generic  
parameters are declared with impl and some are declared with the method  
definition. Here, the generic parameters X1 and Y1 are declared after impl  
because they go with the struct definition. The generic parameters X2 and Y2 are  
declared after fn mixup, because they’re only relevant to the method.



### Performance of Code Using Generics

You might be wondering whether there is a runtime cost when using generic type  
parameters. The good news is that using generic types won't make your program  
run any slower than it would with concrete types.

Rust accomplishes this by performing monomorphization of the code using generics  
at compile time. Monomorphization is the process of turning generic code into  
specific code by filling in the concrete types that are used when compiled. In  
this process, the compiler does the opposite of the steps we used to create  
the generic function in Listing 10-5: the compiler looks at all the places where  
generic code is called and generates code for the concrete types the generic  
code is called with.

Let’s look at how this works by using the standard library’s generic Option<T>  
enum:

```rust
let integer = Some(5);
let float = Some(5.0);
```

When Rust compiles this code, it performs monomorphization. During that process,  
the compiler reads the values that have been used in Option<T> instances and  
identifies two kinds of Option<T>: one is i32 and the other is f64. As such, it  
expands the generic definition of Option<T> into two definitions specialized to  
i32 and f64, thereby replacing the generic definition with the specific ones.

The monomorphized version of the code looks similar to the following (the  
compiler uses different names than what we’re using here for illustration):

> Filename: src/main.rs

In [43]:
#[derive(Debug)]
enum Option_i32{
    Some(i32),
    None
}

#[derive(Debug)]
enum Option_f64{
    Some(f64),
    None
}

{
    let integer = Option_i32::Some(5);
    let float = Option_f64::Some(5.0);

    println!("integer = {:?} float = {:?}", integer, float);
}

integer = Some(5) float = Some(5.0)


()

The generic Option<T> is replaced with the specific definitions created by the  
compiler. Because Rust compiles generic code into code that specifies the type  
in each instance, we pay no runtime cost for using generics. When the code runs,  
it performs just as it would if we had duplicated each definition by hand. The  
process of monomorphization makes Rust’s generics extremely efficient at  
runtime.